# 2020-nCoV Global Cases by Wooil Jeong

- **Wooil Jeong**  
[Dashboard by WooilJeong](https://plot.ly/dashboard/coronavirus:34/present#/)  
[Blog](https://wooiljeong.github.io/etc/corona_dash/)  
[Github Repository](https://github.com/WooilJeong/novel_coronavirus)  


- **Novel Coronavirus (2019-nCoV) Cases, provided by JHU CSSE**  
[Dashboard by JHU CSSE](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)  
[Old Data Sheets](https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/htmlview?usp=sharing&sle=true#)  
[New Google Sheet Link (support comments)](https://docs.google.com/spreadsheets/d/1wQVypefm946ch4XDp37uZ-wartW4V7ILdg-qYiDXUHM/edit?usp=sharing)  
[Time series google sheet](https://docs.google.com/spreadsheets/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/edit?usp=sharing)  


- **Contact**  
email : wooil@kakao.com  

## Create Visualization

## Libraries

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chart_studio
plotly_key=open(".plotly_key").read()
chart_studio.tools.set_credentials_file(username='coronavirus', api_key=plotly_key)
import chart_studio.plotly as py
import plotly.express as px

## Load Dataset

### 프로젝트 디렉토리로 설정

In [2]:
# os.chdir('/Users/wooil/github/novel_coronavirus/')
os.getcwd()

'/Users/wooil/github/novel_coronavirus'

### 데이터가 저장된 디렉토리 내 파일 목록 조회

In [3]:
file_list = glob.glob("Data/*")
file_list = sorted(file_list, reverse=True)
file_list

['Data/Dataset_20200210_1009.csv', 'Data/Dataset_20200210_0946.csv']

### 최신 데이터 불러오기

In [4]:
try:
    df = pd.read_csv(file_list[0])
except:
    df = pd.read_csv(file_list[0].replace('\\','/')+'.csv')

## 전 세계 통합 데이터

In [5]:
def agg(df):
    '''
    국가 및 지역 별 데이터 집계 함수
    '''

    df_region = df.groupby('Last Update').agg(
        {'Confirmed':['sum'], 'Death':['sum'],'Recovered':['sum']}).reset_index()
    
    df_region.index = range(len(df_region))
    df_region.columns=['Last Update', 'Confirmed', 'Death', 'Recovered']
    
    # Feature Engineering
    df_region['D/C'] = (df_region['Death']/df_region['Confirmed'])*100
    df_region['R/C'] = (df_region['Recovered']/df_region['Confirmed'])*100
    
    return df_region

In [6]:
df_all = agg(df)
df_all.tail()

,Last Update,Confirmed,Death,Recovered,D/C,R/C
29,2020-02-06 14:20:00,28353,565,1382,1.992734,4.874264
30,2020-02-07 20:13:00,31532,638,1763,2.023341,5.591146
31,2020-02-07 22:50:00,34899,724,1995,2.074558,5.716496
32,2020-02-08 23:04:00,37549,813,2701,2.165171,7.193267
33,2020-02-09 10:30:00,37592,814,2922,2.165354,7.772930


## Map

In [7]:
df.head()

,Province/State,Country/Region,Lat,Long,Last Update,Confirmed,Death,Recovered,D/C,R/C
0,"Seattle, WA",US,47.75110,-120.7400,2020-01-21 22:00:00,1,0,0,0.0,0.000000
1,Hubei,Mainland China,30.97564,112.2707,2020-01-21 22:00:00,270,0,25,0.0,9.259259
2,Henan,Mainland China,33.88202,113.6140,2020-01-21 22:00:00,0,0,0,0.0,0.000000
3,Heilongjiang,Mainland China,47.86200,127.7622,2020-01-21 22:00:00,0,0,0,0.0,0.000000
4,Hainan,Mainland China,19.19673,109.7455,2020-01-21 22:00:00,0,0,0,0.0,0.000000


In [8]:
# px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(df, 
                        title='World New Coronavirus Chart',
                        lat="Lat", 
                        lon="Long", 
                        center={'lat':31, 'lon':112},
                        color="Country/Region", 
                        size="Confirmed",
                        hover_name="Province/State",
                        animation_frame= "Last Update",
                        animation_group ="Province/State",
                        color_continuous_scale = px.colors.cyclical.IceFire, 
                        template='plotly_dark', 
                        size_max=75, 
                        zoom=3)

fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona_scatter')

## 전 세계 통합 확진자, 사망자, 회복자 추이

In [9]:
data = df_all.copy()
data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])

fig = px.line(data, x="Last Update", y="Confirmed", template='plotly_dark', title='Total Confirmed', text='Confirmed')
fig2= px.line(data_melt, x="Last Update", y="value", color='variable', template='plotly_dark', title='Total Death and Recovered', text='value')

In [10]:
fig.update_traces(textposition='top center')
fig.update_layout(legend_orientation="h")

py.iplot(fig, filename='corona_confirmed')

In [11]:
fig2.update_traces(textposition='top center')
fig2.update_layout(legend_orientation="h")

py.iplot(fig2, filename='corona_death_recovered')

## 국가 및 지역 별 데이터프레임 생성

### 리스트 생성

In [12]:
# 모든 국가 리스트
country_list = list(set(df['Country/Region']))

# 제외할 국가 리스트(제외 국가는 세부 지역 별 리스트 생성하여 분석)
except_list = ['Mainland China', 'US', 'Canada', 'Australia']

# 세계 국가 리스트 재정의(일부 국가 제외)
world_list = country_list.copy()
for i in except_list:
    world_list.remove(i)

# 일부 국가(세부 지역 별 리스트 생성)
## 중국 지역 리스트
china_list = list(set(df[df['Country/Region']=='Mainland China']['Province/State']))
### 후베이 제외 중국 지역 리스트
china_list2 = china_list.copy()
china_list2.remove('Hubei')

## 미국 지역 리스트
us_list = list(set(df[df['Country/Region']=='US']['Province/State']))

## 캐나다 지역 리스트
canada_list = list(set(df[df['Country/Region']=='Canada']['Province/State']))

## 호주 지역 리스트
australia_list = list(set(df[df['Country/Region']=='Australia']['Province/State']))

### 데이터프레임 생성

In [13]:
for i in country_list:
    globals()[i.replace(' ','_')] = df[df['Country/Region']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))
     
for i in china_list:
    globals()[i.replace(' ','_')] = Mainland_China[Mainland_China['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))
    
for i in us_list:
    globals()[i.replace(' ','_')] = US[US['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))
    
for i in australia_list:
    globals()[i.replace(' ','_')] = Australia[Australia['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))
    
for i in canada_list:
    globals()[i.replace(' ','_')] = Canada[Canada['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))

Finland
Mainland_China
Hong_Kong
France
Belgium
Italy
UK
Russia
Malaysia
Nepal
Japan
Macau
Germany
Sweden
Spain
United_Arab_Emirates
Singapore
Sri_Lanka
Cambodia
US
Canada
Philippines
South_Korea
Taiwan
Thailand
Australia
Vietnam
India
Others
Santa_Clara,_CA
Chicago,_IL
Boston,_MA
Seattle,_WA
Madison,_WI
San_Benito,_CA
Orange,_CA
Tempe,_AZ
Los_Angeles,_CA
Victoria
Queensland
South_Australia
New_South_Wales
Toronto,_ON
London,_ON
British_Columbia


In [ ]:
x_max = int(np.max(Mainland_China['Recovered']) * 1.2)
y_max = int(np.max(Mainland_China['Death']) * 1.2)

fig = px.scatter(Mainland_China, 
               x="Recovered", 
               y="Death",
               color='Province/State', 
               title='Chinese New Coronavirus Chart',
               template='plotly_dark', 
               log_x= False,                
               animation_frame= "Last Update",
               animation_group ="Province/State",
               hover_name='Province/State', 
               range_x = [-50,x_max], 
               range_y = [-50,y_max],
               size='Confirmed',
               size_max=50,
               labels = dict(Death='Death',
                             Recovered = 'Recovered'))

fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona')

## 전 세계 비교 그래프

In [15]:
time_list = list(set(df['Last Update']))
time_list = pd.DataFrame(time_list, columns=['Last Update']).sort_values('Last Update')
time_list.index = range(len(time_list))

total_list=world_list+china_list+us_list+australia_list+canada_list

for i in total_list:
    i=i.replace(' ','_')
    time_list=pd.merge(time_list, globals()[i][['Last Update', 'Confirmed']], how='left', on='Last Update')
time_list.columns=['Last Update']+total_list
time_list=time_list.fillna(0)

df_world = time_list[['Last Update']+world_list]
df_china = time_list[['Last Update']+china_list]
df_china2 = time_list[['Last Update']+china_list2]
df_us = time_list[['Last Update']+us_list]
df_australia = time_list[['Last Update']+australia_list]
df_canada = time_list[['Last Update']+canada_list]

## 전 세계

In [16]:
data = df_world.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=world_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed by country',
            template='plotly_dark')

fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona_world')

## 중국

In [17]:
data = df_china.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=china_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (China)',
            template='plotly_dark')

fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona_china')

## 중국(후베이 제외)

In [18]:
data = df_china2.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=china_list2)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (China without Hubei)',
            template='plotly_dark'
           )
fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona_china_without_hubei')

## 미국

In [19]:
data = df_us.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=us_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (US)',
            template='plotly_dark'
           )
fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona_us')

## 호주

In [20]:
data = df_australia.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=australia_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (Australia)',
            template='plotly_dark'
           )
fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona_australia')

## 캐나다

In [21]:
data = df_canada.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=canada_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (Canada)',
            template='plotly_dark'
           )
fig.update_layout(showlegend=False)
py.iplot(fig, filename='corona_canada')

## 개별 국가 시각화

## World

In [22]:
# for i in world_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## Mainland China

In [23]:
# for i in china_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## US

In [24]:
# for i in us_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## Australia

In [25]:
# for i in australia_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## Canada

In [26]:
# for i in canada_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()